<a href="https://colab.research.google.com/github/Meta-Sean/Practical-Deep-Learning/blob/main/corncomp_pt3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Sep 22 18:06:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install kaggle fastkaggle fastai timm>=0.6.2.dev0 pynvml

In [ ]:
from google.colab import files
import pandas as pd
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
! mkdir ~/.kaggle #creating folder
! cp kaggle.json ~/.kaggle/ #copying kaggle.json
! chmod 600 ~/.kaggle/kaggle.json #reading the file with full access

In [ ]:
from fastkaggle import * 
from fastai.vision.all import *

comp = 'kaggle-pog-series-s01e03'

path = setup_comp(comp, install='"fastcore>=1.4.5" "fastai>=2.7.1" "timm>=0.6.2.dev0"')

100%|██████████| 491M/491M [00:01<00:00, 258MB/s]

In [ ]:
from pathlib import Path
path = Path('/content/kaggle-pog-series-s01e03/corn/')
path

Path('/content/kaggle-pog-series-s01e03/corn')

In [ ]:
trn_path = path/'train'
files = get_image_files(trn_path)
tst_files = get_image_files(path/'test').sorted()

In [ ]:
train_df = pd.read_csv(path/'train.csv')
train_df.head()

,seed_id,view,image,label
0,0,top,train/00000.png,broken
1,1,bottom,train/00001.png,pure
2,3,top,train/00003.png,broken
3,4,top,train/00004.png,pure
4,5,top,train/00005.png,discolored


In [ ]:
df = pd.read_csv(path/'train.csv')
df.label.value_counts()

pure          5837
broken        4554
discolored    2504
silkcut       1427
Name: label, dtype: int64

In [ ]:
def train(arch, size, item=Resize(132, method='squish'), finetune=True, epochs=12):
    dls = ImageDataLoaders.from_df(train_df, path=path, valid_pct=0.2, label_col=3, fn_col=2, item_tfms=item,
                                   batch_tfms=aug_transforms(size=size, min_scale=0.75))
    #cbs = GradientAccumulation(64) if accum else []
    learn = vision_learner(dls, arch, metrics=[error_rate, accuracy]).to_fp16()
    if finetune:
        learn.fine_tune(epochs, 0.01)
        return learn.tta(dl=dls.test_dl(tst_files))
    else:
        learn.unfreeze()
        learn.fit_one_cycle(epochs, 0.01)


In [ ]:
import gc
def report_gpu():
    print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
report_gpu()

GPU:0
no processes are running


In [ ]:
trn_path = path/'train'

In [ ]:
#train('convnext_small_in22k', 128, epochs=1, accum=2, finetune=False)
#report_gpu()

In [ ]:
res = (132,128)

In [ ]:
models = {
    'convnext_base_in22k': {
        (Resize(res), (320,224)),
    }, 'beit_base_patch16_224': {
        (Resize(132, method='squish'), 224),
        (Resize(res), 224),
    }, 'swin_base_patch4_window7_224': {
        (Resize(res), 224),
    }
}

In [ ]:
tta_res = []

for arch,details in models.items():
    for item,size in details:
        print('---',arch)
        print(size)
        print(item.name)
        tta_res.append(train(arch, size, item=item)) #, epochs=1))
        gc.collect()
        torch.cuda.empty_cache()

--- convnext_base_in22k
(320, 224)
Resize -- {'size': (128, 132), 'method': 'crop', 'pad_mode': 'reflection', 'resamples': (2, 0), 'p': 1.0}


epoch,train_loss,valid_loss,error_rate,accuracy,time
0,1.014952,0.780048,0.301676,0.698324,05:54


epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.679110,0.613874,0.247556,0.752444,12:10
1,0.610606,0.616105,0.239874,0.760126,12:10
2,0.590613,0.615959,0.241620,0.758380,12:10
3,0.526038,0.607446,0.235684,0.764316,12:10


In [ ]:
details = [(Resize(132, method='squish'),112),(Resize(240, method='squish'),112),(Resize(132),112)]
    

In [ ]:
arch = 'convnext_small_in22k'

In [ ]:
tta_res = []

for item,size in details:
    print('---',arch)
    print(size)
    print(item.name)
    tta_res.append(train(arch, size, item=item, accum=2)) #, epochs=1))
    gc.collect()
    torch.cuda.empty_cache()

--- convnext_small_in22k
112
Resize -- {'size': (132, 132), 'method': 'squish', 'pad_mode': 'reflection', 'resamples': (2, 0), 'p': 1.0}


Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_small_22k_224.pth" to /root/.cache/torch/hub/checkpoints/convnext_small_22k_224.pth


epoch,train_loss,valid_loss,error_rate,accuracy,time
0,1.040065,0.937423,0.321927,0.678073,00:39


epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.695516,0.583678,0.241969,0.758031,00:53
1,0.622451,0.579245,0.234986,0.765014,00:53
2,0.595904,0.554265,0.222765,0.777235,00:53
3,0.607642,0.546338,0.214036,0.785964,00:52
4,0.547375,0.585204,0.234637,0.765363,00:52
5,0.510140,0.626182,0.240223,0.759777,00:52
6,0.413624,0.591897,0.211592,0.788408,00:53
7,0.368843,0.638979,0.218575,0.781425,00:52
8,0.305921,0.635765,0.216830,0.783170,00:52
9,0.224268,0.697281,0.217528,0.782472,00:52


--- convnext_small_in22k
112
Resize -- {'size': (240, 240), 'method': 'squish', 'pad_mode': 'reflection', 'resamples': (2, 0), 'p': 1.0}


epoch,train_loss,valid_loss,error_rate,accuracy,time
0,1.108487,0.795948,0.292947,0.707053,00:38


epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.709316,0.562413,0.230447,0.769553,00:53
1,0.636620,0.536780,0.213687,0.786313,00:54
2,0.587088,0.555027,0.224162,0.775838,00:54
3,0.587019,0.586235,0.243366,0.756634,00:54
4,0.535428,0.563291,0.219623,0.780377,00:54
5,0.525989,0.679405,0.245461,0.754539,00:53
6,0.409254,0.567954,0.213687,0.786313,00:54
7,0.383769,0.578377,0.215433,0.784567,00:54
8,0.299618,0.649040,0.218226,0.781774,00:54
9,0.256037,0.750743,0.238827,0.761173,00:54


--- convnext_small_in22k
112
Resize -- {'size': (132, 132), 'method': 'crop', 'pad_mode': 'reflection', 'resamples': (2, 0), 'p': 1.0}


epoch,train_loss,valid_loss,error_rate,accuracy,time
0,1.085151,0.899807,0.320880,0.679120,00:35


epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.715220,0.615012,0.245810,0.754190,00:52
1,0.610024,0.593192,0.236732,0.763268,00:51
2,0.602242,0.591238,0.229749,0.770251,00:52
3,0.598425,0.555719,0.218575,0.781425,00:52
4,0.546897,0.562751,0.229050,0.770950,00:51
5,0.498633,0.585600,0.223813,0.776187,00:52
6,0.421491,0.624712,0.235335,0.764665,00:52
7,0.371704,0.672722,0.235684,0.764316,00:52
8,0.290579,0.724311,0.231145,0.768855,00:52
9,0.262264,0.784417,0.238128,0.761872,00:52


In [ ]:
save_pickle('tta_res.pkl', tta_res)

In [ ]:
tta_prs = first(zip(*tta_res))

In [ ]:
tta_res

In [ ]:
avg_pr = torch.stack(tta_prs).mean(0)
avg_pr.shape

In [ ]:
dls = ImageDataLoaders.from_csv(path, 'train.csv', fn_col=2, label_col=3, item_tfms=Resize(132, method='squish'),
                                   batch_tfms=aug_transforms(size=128, min_scale=0.75))

In [ ]:
idxs = avg_pr.argmax(dim=1)
vocab = np.array(dls.vocab)


In [ ]:
avg_pr

In [ ]:
ss = pd.read_csv(path/'sample_submission.csv')
ss = ss['seed_id']
tst_df = pd.read_csv(path/'test.csv')
tst_df
ss

In [ ]:
tst_df['label'] = vocab[idxs]
tst_df

In [ ]:
ss = pd.merge(ss, tst_df, on="seed_id")
ss

In [ ]:
ss = ss[["seed_id", "label"]]
ss

In [ ]:
ss.to_csv('submission.csv', index=False)
!head submission.csv